# Company Brochure Builder

In [ ]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from urllib.parse import urljoin

from utils import fetch_website_contents, fetch_website_links

load_dotenv(override=True)

api_key = os.getenv('OPENROUTER_API_KEY')
base_url = os.getenv('OPENROUTER_BASE_URL', 'https://openrouter.ai/api/v1')
MODEL = os.getenv('OPENROUTER_MODEL', 'openai/gpt-4o-mini')
client = OpenAI(base_url=base_url, api_key=api_key)

In [ ]:
link_prompt = """Select relevant links for a company brochure (About, Careers, Company pages).
Respond in JSON: {"links": [{"type": "about page", "url": "https://full.url/about"}]}"""

def select_relevant_links(url):
    links = fetch_website_links(url)
    prompt = f"Website: {url}\nSelect relevant links (exclude Terms, Privacy, email).\n\nLinks:\n" + "\n".join(links)
    
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_prompt},
            {"role": "user", "content": prompt}
        ],
        response_format={"type": "json_object"}
    )
    
    result = json.loads(response.choices[0].message.content)
    for link in result.get('links', []):
        if not link['url'].startswith(('http://', 'https://')):
            link['url'] = urljoin(url, link['url'])
    return result

In [ ]:
def fetch_all_content(url):
    content = f"## Landing Page:\n\n{fetch_website_contents(url)}\n\n## Relevant Pages:\n"
    links = select_relevant_links(url)
    
    for link in links.get('links', []):
        try:
            content += f"\n\n### {link['type']}\n{fetch_website_contents(link['url'])}"
        except:
            content += f"\n\n### {link['type']}\n[Content unavailable]"
    
    return content[:5000]

In [ ]:
brochure_prompt = """Create a professional brochure from company website content.
Include company culture, customers, and careers. Respond in markdown without code blocks."""

def create_brochure(company_name, url, stream=False):
    prompt = f"Company: {company_name}\n\n{fetch_all_content(url)}"
    
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_prompt},
            {"role": "user", "content": prompt}
        ],
        stream=stream
    )
    
    if stream:
        result = ""
        handle = display(Markdown(""), display_id=True)
        for chunk in response:
            if chunk.choices[0].delta.content:
                result += chunk.choices[0].delta.content
                update_display(Markdown(result), display_id=handle.display_id)
        return result
    else:
        result = response.choices[0].message.content
        display(Markdown(result))
        return result

In [ ]:
create_brochure("HuggingFace", "https://huggingface.co")
# create_brochure("Company Name", "https://company.com", stream=True)